# Tasks with genomic DBs

## Parsing CSV files from individual genes in gnomAD

Example: [file obtained for GPX2:](https://gnomad.broadinstitute.org/gene/ENSG00000176153)

```csv
Chromosome,Position,rsIDs,Reference,Alternate,Source,Filters - exomes,Filters - genomes,Transcript,HGVS Consequence,Protein Consequence,Transcript Consequence,VEP Annotation,ClinVar Clinical Significance,ClinVar Variation ID,Flags,Allele Count,Allele Number,Allele Frequency,Homozygote Count,Hemizygote Count,Allele Count African/African American,Allele Number African/African American,Homozygote Count African/African American,Hemizygote Count African/African American,Allele Count Latino/Admixed American,Allele Number Latino/Admixed American,Homozygote Count Latino/Admixed American,Hemizygote Count Latino/Admixed American,Allele Count Ashkenazi Jewish,Allele Number Ashkenazi Jewish,Homozygote Count Ashkenazi Jewish,Hemizygote Count Ashkenazi Jewish,Allele Count East Asian,Allele Number East Asian,Homozygote Count East Asian,Hemizygote Count East Asian,Allele Count European (Finnish),Allele Number European (Finnish),Homozygote Count European (Finnish),Hemizygote Count European (Finnish),Allele Count European (non-Finnish),Allele Number European (non-Finnish),Homozygote Count European (non-Finnish),Hemizygote Count European (non-Finnish),Allele Count Other,Allele Number Other,Homozygote Count Other,Hemizygote Count Other,Allele Count South Asian,Allele Number South Asian,Homozygote Count South Asian,Hemizygote Count South Asian
14,65406155,rs1471966487,G,T,gnomAD Exomes,PASS,NA,ENST00000389614.5,c.*51C>A,,c.*51C>A,3_prime_UTR_variant,,,,1,218184,0.0000045832875004583285,0,0,0,15192,0,0,0,31042,0,0,0,6818,0,0,1,17494,0,0,0,19274,0,0,0,100426,0,0,0,5126,0,0,0,22812,0,0
14,65406166,rs1418718413,C,T,gnomAD Exomes,PASS,NA,ENST00000389614.5,c.*40G>A,,c.*40G>A,3_prime_UTR_variant,,,,1,229576,0.00000435585601282364,0,0,0,15322,0,0,0,32902,0,0,0,7906,0,0,1,17834,0,0,0,20042,0,0,0,104758,0,0,0,5506,0,0,0,25306,0,0
(...)
```

## Querying ENSEMBL

In [1]:
#!pip install pandas
import pandas as pd
from pathlib import Path

# define the directory where I keep the data
datadir = Path("""./input""")
filename=datadir/"gnomAD_v2.1.1_ENSG00000176153_2023_09_24_12_56_03.csv"


# load the CSV file
data=pd.read_csv(filename)
# print(data)
data.loc[:,["Protein Consequence","ClinVar Clinical Significance"]]
data = data[data["Protein Consequence"].notna()].loc[:,["Protein Consequence","ClinVar Clinical Significance"]]
data[data["ClinVar Clinical Significance"].notna()]

#contain_values = df[df[

,Protein Consequence,ClinVar Clinical Significance
64,p.Lys138Arg,Uncertain significance
237,p.Arg56His,Uncertain significance


In [2]:
from Bio import Entrez
import sys

# uncomment and add your e-mail address:
# Entrez.email="me@uvic.cat"

list_of_genes=['GPX1']
term_entrez=' OR '.join(list_of_genes)

print('querying for... ',term_entrez)

esearch_result = Entrez.esearch(db="gene",term='GPX3',rettype='fasta',retmod="text")
parsed_result = Entrez.read(esearch_result)
print(parsed_result)

request = Entrez.epost("gene",id=",".join(parsed_result['IdList']))
try:
   result = Entrez.read(request)
except RuntimeError as e:
   print("An error occurred while retrieving the annotations.")
   print("The error returned was %s" % e)
   sys.exit(-1)
 
webEnv = result["WebEnv"]
queryKey = result["QueryKey"]
efetch_result = Entrez.efetch(db="gene", webenv=webEnv, query_key = queryKey, retmode="xml")
genes = Entrez.read(efetch_result)
print(genes)


querying for...  GPX1


/Users/jordivilla/miniconda3/lib/python3.11/site-packages/Bio/Entrez/__init__.py:658: UserWarning: 
Email address is not specified.

To make use of NCBI's E-utilities, NCBI requires you to specify your
email address with each request.  As an example, if your email address
is A.N.Other@example.com, you can specify it as follows:
   from Bio import Entrez
   Entrez.email = 'A.N.Other@example.com'
In case of excessive usage of the E-utilities, NCBI will attempt to contact
a user at the email address provided before blocking access to the
E-utilities.
  warnings.warn(


{'Count': '623', 'RetMax': '20', 'RetStart': '0', 'IdList': ['132235860', '132080472', '7124', '3569', '3091', '5743', '3576', '5468', '4780', '5444', '2952', '2950', '4233', '2908', '13982', '335', '155871', '6648', '10135', '860'], 'TranslationSet': [], 'TranslationStack': [{'Term': 'GPX3[All Fields]', 'Field': 'All Fields', 'Count': '623', 'Explode': 'N'}, 'GROUP'], 'QueryTranslation': 'GPX3[All Fields]'}


KeyboardInterrupt: 